In [1]:
#-----------------------------------------
# importamos los paquetes que vamos a usar
#-----------------------------------------

import numpy as np
import glob                         # para hacer listas de archivo
import subprocess                   # para usar subprocesos

In [41]:
#-----------------------------------------
# nos quedamos con los .pfds de nbin=1024
#-----------------------------------------

# creamos una lista con todos los archivos .pfd

timing_folder = './A1/'
pfds = glob.glob(timing_folder+'*pfd')                   # lista de archivos .pfd
flen = len(pfds[1])                                      # longitud del nombre de un archivo .pfd
pfds_1024 = np.empty(len(pfds), dtype="S"+str(flen))     # lista en la que guardaremos los .pfd con nbin=1024
pfds_1024_128 = np.empty(len(pfds), dtype="S"+str(flen)) # lista en la que guardaremos los .pfd con nbin=1024 y nchan=128
n=0                                                      # numero de archivos .pfd con nbin=1024
m=0                                                      # numero de archivos .pfd con nbin=1024 y nchan=128
output = "output.dat"                                    # nombre del archivo de salida del cual vamos leyendo los headers

res = open("results.dat", "w")                           # archivo con los resultados

# para cada archvio .pfd, generamos un archivo con su header y leemos su nbins y su nchan

for i in range(len(pfds)):

    subprocess.call( "psrstat >" + output + " " + pfds[i] , shell=True )
    
    n_bins = np.genfromtxt ( output, comments="none", dtype=int, skip_header=1, max_rows=1, usecols=(6) )
    n_chan = np.genfromtxt ( output, comments="none", dtype=int, skip_header=2, max_rows=1, usecols=(5) )
    
    if n_bins==1024:                                    # si el .pfd tiene nbin=1024
        pfds_1024[n] = pfds[i]                          # entonces lo guardamos en la lista
        n += 1                                          # y aumentamos en 1 el numero de pfds con nbin=1024
        
        if n_chan==128:                                 # si el .pfd tiene nchan=1024           
            pfds_1024_128[m] = pfds[i]                  # entonces lo guardamos en la lista
            m += 1                              

In [44]:
#-----------------------------------------------------------
# generamos el template usando los archivos enlistados antes
#-----------------------------------------------------------

nbin=1024

# primero, generamos un string que contenga todos los archivos .pfd
# de nchan=128, con los que vamos a generar el template

archivos128 = ''
for j in range(m):
    archivos128 = archivos128 + str(pfds_1024_128[j].decode('utf-8')) + ' '

# ahora llamamos al psradd para generar el template sumando observaciones

template = "template_" + str(nbin) + ".std"   # nombre del template

subprocess.call( "psradd -P " + str(archivos128) + "-o " + str(template) , shell=True )

0

In [38]:
#-----------------------------------------
# generamos los TOAs (el archivo .tim)
#-----------------------------------------

# primero, generamos un string que contenga todos los archivos .pfd

archivos = ''
for j in range(n):
    archivos = archivos + str(pfds_1024[j].decode('utf-8')) + ' '
    
# ahora llamamos a pat para generar el archivo .tim

timing_file = "timing_" + str(nbin) + ".tim"   # nombre del archivo .tim

subprocess.call( "pat -FT -A PGS -s " + template + " -f >" + timing_file + " 'tempo2' " + archivos , shell=True )

255

In [ ]:
#-----------------------------------------
# calculamos residuos usando tempo2
#-----------------------------------------

residue_file = "residues_" + str(nbin) + ".tim"   # nombre del archivo de los residuos

subprocess.call( "tempo2 >" + residue_file + " -us -f J0437-4715.par " + timing_file , shell=True)

# abrimos el archivo generado por tempo2, donde se encuentra el RMS que buscamos

    RMS = np.genfromtxt ( residue_file, comments="none", dtype=float, skip_header=18, max_rows=1, usecols=(10) )
    
# escribimos el resultado en el archivo de salida

res.write( nbin, RMS )